In [ ]:
# Install dependencies
!pip install -q llama-index PyMuPDF sentence-transformers faiss-cpu transformers

# Imports
from google.colab import files
import fitz  # PyMuPDF
import numpy as np
import faiss
import torch
from sentence_transformers import SentenceTransformer
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 18.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 26.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 62.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 267.3/267.3 kB 15.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.0/41.0 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 63.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 51.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 22.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.

In [ ]:
# Upload multiple PDFs
uploaded = files.upload()

# Extract text from PDFs
def extract_text_from_pdf(pdf_path):
    doc = fitz.open(pdf_path)
    text = ""
    for page in doc:
        text += page.get_text()
    return text

# Combine text from all uploaded PDFs
all_text = ""
for filename in uploaded.keys():
    all_text += extract_text_from_pdf(filename) + "\n"

Saving Contractual_condition_for_contract-related_travel_20210701.pdf to Contractual_condition_for_contract-related_travel_20210701.pdf
Saving Siemens Travel policy.pdf to Siemens Travel policy.pdf


In [ ]:
 #split into chunks
def split_text(text, max_length=500):
    return [text[i:i+max_length] for i in range(0, len(text), max_length)]

chunks = split_text(all_text)
# Print the chunks
for i, chunk in enumerate(chunks):
    print(f"Chunk {i}:\n{chunk}\n{'-'*40}")


Chunk 0:
Contractual condition for contract-related travel costs. Version dated: July 1, 2021                    Page 1 of 3  
CSD 2.3 | 15 years from CSD-event , maximum of 99 years from creation 
BA-M 
PUBLIC 
 
Contractual condition for contract-related travel 
costs 
Version dated: July 1, 2021 
 
1.1 
Subject of the regulation 
These contractual conditions apply only to business travel to a location other than the location 
where services are provided on a permanent or contractually agreed basis. Th
----------------------------------------
Chunk 1:
ese conditions 
shall apply in addition to the respective agreed procurement conditions of Volkswagen AG 
(hereinafter referred to as the Client) and shall regulate commissioning, performing and 
invoicing business travel carried out on the basis of these terms and conditions by the 
contractor within the scope of the contractual relationship. Deviations from these contractual 
conditions, which have not been discussed with the Client a

In [ ]:
# Sentence embedding
embedding_model = SentenceTransformer('all-MiniLM-L6-v2')
embeddings = embedding_model.encode(chunks)

# FAISS index
dimension = embeddings.shape[1]
index = faiss.IndexFlatL2(dimension)
index.add(np.array(embeddings))

# Chunk mapping
chunk_mapping = {i: chunk for i, chunk in enumerate(chunks)}

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
# Load TinyLLaMA
llama_model_name = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"
tokenizer = AutoTokenizer.from_pretrained(llama_model_name)
llama_model = AutoModelForCausalLM.from_pretrained(llama_model_name)

# Text generation pipeline
generator = pipeline("text-generation", model=llama_model, tokenizer=tokenizer, device=0 if torch.cuda.is_available() else -1)

# Retrieve relevant chunks
def retrieve_relevant_chunks(query, top_k=4):
    query_embedding = embedding_model.encode([query])
    distances, indices = index.search(np.array(query_embedding), top_k)
    return [chunk_mapping[i] for i in indices[0]]

tokenizer_config.json:   0%|          | 0.00/1.29k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/551 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/608 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.20G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

Device set to use cpu


In [ ]:
# Generate answer
def generate_answer(query):
    context_chunks = retrieve_relevant_chunks(query)
    context = "\n".join(context_chunks)
    prompt = f"Context:\n{context}\n\nQuestion: {query}\nAnswer:"
    output = generator(prompt, max_new_tokens=200, do_sample=True)[0]['generated_text']
    return output

In [ ]:
# Example query
print(generate_answer("Who is the client? and what is it related to?"))

Context:

The contractor – with the prior written consent of the Client’s relevant department – books the 
travel service independently and invoices the Client for the net costs incurred using a proper 
invoice within the meaning of Section 14 Umsatzsteuergesetz (UStG, German VAT Act) 
without adding any mark-up. 
1.3.2 Business travel as an integral part of the contract 
The travel costs within a defined scope are already a fixed and essential part of the contract 
and have been defined accordingly ther
ese conditions 
shall apply in addition to the respective agreed procurement conditions of Volkswagen AG 
(hereinafter referred to as the Client) and shall regulate commissioning, performing and 
invoicing business travel carried out on the basis of these terms and conditions by the 
contractor within the scope of the contractual relationship. Deviations from these contractual 
conditions, which have not been discussed with the Client and agreed in writing, will result in 
travel expen

In [ ]:
# Example query
print(generate_answer("What is GSI?"))

In [ ]:
# Example query
print(generate_answer("Can first class flight tickets be booked for business travel"))

In [ ]:
# Example query
print(generate_answer("Who is the Contact person?"))

Context:
able at https://sie.ag/3MGicvA 
Follow us on Twitter at www.twitter.com/siemens_press 
 
Contact for journalists  
Andreas Friedrich 
Phone: +49 1522 2103967 
E-mail: friedrich@siemens.com 
 
Konstanze Somborn 
Phone: +49 173 8947354 
E-mail: konstanze.somborn@siemens.com 
 
 
Siemens AG (Berlin and Munich) is a technology company focused on industry, infrastructure, transport, and 
healthcare. From more resource-efficient factories, resilient supply chains, and smarter buildings and grids, to 

 issued to the contractor (=invoice recipient), must be submitted as a numbered 
copy and must contain the following information: 
1. Full name and full address of the company providing the service (=receipt issuer) 
2. Full name and full address of the recipient of the service (=invoice recipient) 
3. Tax number or VAT identification number of the company providing the service (=receipt 
issuer) 
4. Invoice number 
5. Invoice date 
6. Type and scope of services (e.g. taxi incl. journe